In [2]:
import pandas as pd
import re
import numpy as np
from sklearn.preprocessing import StandardScaler

# Load the CSV file
file_path = 'cars.csv'
df = pd.read_csv(file_path)

# Make a copy of the DataFrame to avoid modifying the original one
df_cleaned = df.copy()

# Define the price format "currency followed by a number"
price_pattern = r'^[A-Z]+\s\d{1,3}(?:,\d{3})*$'

# Check which rows follow the price pattern
df_cleaned['price_matches_pattern'] = df_cleaned['price'].apply(lambda x: bool(re.match(price_pattern, str(x))))
valid_prices_df = df_cleaned[df_cleaned['price_matches_pattern']].copy()

# Extract the currency symbols only from the valid rows
df_cleaned['currency'] = df_cleaned['price'].str.extract(r'([A-Za-z]+)', expand=False)

# Replace invalid price values with NaN
df_cleaned.loc[~df_cleaned['price_matches_pattern'], 'price'] = np.nan

# Define conversion rates (you can adjust these rates)
conversion_rates = {
    'USD': 1,
    'AED': 0.27,
    'SAR': 0.27,
    'EGP': 0.032,
    'BHD': 2.65,
    'QAR': 0.27,
    'OMR': 2.60,
    'KWD': 3.30
}

# Clean and convert price to numeric value
def clean_price(price_str):
    if isinstance(price_str, str):
        # Remove the commas and extract the numeric part
        price_str = re.sub(r'[^\d.]', '', price_str)
        return pd.to_numeric(price_str, errors='coerce')
    return np.nan

df_cleaned['numeric_price'] = df_cleaned['price'].apply(clean_price)

# Function to convert price to USD based on currency
def convert_price_to_usd(row):
    if pd.notna(row['numeric_price']) and row['currency'] in conversion_rates:
        return row['numeric_price'] * conversion_rates[row['currency']]  # Convert to USD
    return np.nan

# Apply the conversion function to the DataFrame
df_cleaned['price_in_usd'] = df_cleaned.apply(convert_price_to_usd, axis=1)

# Fill NaN values in price_in_usd with the mean of the column
df_cleaned['price_in_usd'] = df_cleaned['price_in_usd'].fillna(df_cleaned['price_in_usd'].mean())

# Process engine_capacity column
df_cleaned['engine_capacity'] = pd.to_numeric(df_cleaned['engine_capacity'], errors='coerce')
df_cleaned['engine_capacity'] = np.where(df_cleaned['engine_capacity'] > 10, df_cleaned['engine_capacity'] / 1000, df_cleaned['engine_capacity'])
df_cleaned['engine_capacity'] = df_cleaned['engine_capacity'].apply(lambda x: x if 0.5 <= x <= 10 else np.nan)
engine_capacity_mean = df_cleaned['engine_capacity'].mean()
df_cleaned['engine_capacity'] = df_cleaned['engine_capacity'].fillna(engine_capacity_mean).astype(int)

# Clean the 'cylinder' column
def clean_cylinder(cylinder):
    if isinstance(cylinder, str):
        match = re.search(r'\d+', cylinder)
        return int(match.group()) if match else np.nan
    return np.nan

df_cleaned['cylinder'] = df_cleaned['cylinder'].apply(clean_cylinder)
cylinder_mean = df_cleaned['cylinder'].mean()
df_cleaned['cylinder'] = df_cleaned['cylinder'].fillna(cylinder_mean).astype(int)

# Clean the 'horse_power' column
def clean_horsepower(hp):
    if isinstance(hp, str):
        match = re.search(r'^\d+$', hp)
        if match:
            return int(match.group())
        else:
            return np.nan
    return hp

df_cleaned['horse_power'] = df_cleaned['horse_power'].apply(clean_horsepower)
valid_hp_range = (50, 1000)
df_cleaned['horse_power'] = df_cleaned['horse_power'].apply(
    lambda x: np.nan if pd.isna(x) or not valid_hp_range[0] <= x <= valid_hp_range[1] else x
)
horse_power_mean = df_cleaned['horse_power'].mean()
df_cleaned['horse_power'] = df_cleaned['horse_power'].fillna(horse_power_mean).astype(int)

# Clean the 'top_speed' column
def clean_top_speed(speed):
    if isinstance(speed, str):
        if re.search(r'(\d+\s*seater|N\s*A|[^0-9])', speed, re.IGNORECASE):
            return np.nan
        cleaned_speed = re.sub(r'[^\d.]', '', speed)
        if cleaned_speed.isdigit():
            return int(cleaned_speed)
        else:
            return np.nan
    return speed

df_cleaned['top_speed'] = df_cleaned['top_speed'].apply(clean_top_speed)
top_speed_mean = df_cleaned['top_speed'].mean()
df_cleaned['top_speed'] = df_cleaned['top_speed'].fillna(top_speed_mean).astype(int)

# Clean the 'seats' column
def clean_seats(seats):
    if isinstance(seats, str):
        match = re.match(r'(\d+)\s*seater', seats, re.IGNORECASE)
        if match:
            return int(match.group(1))
        else:
            return np.nan
    return np.nan

df_cleaned['seats'] = df_cleaned['seats'].apply(clean_seats)
seats_mean = df_cleaned['seats'].mean()
df_cleaned['seats'] = df_cleaned['seats'].fillna(seats_mean).astype(int)

# Standardization of numeric columns
numeric_columns = ['price_in_usd', 'engine_capacity', 'cylinder', 'horse_power', 'top_speed', 'seats']
scaler = StandardScaler()
df_cleaned[numeric_columns] = scaler.fit_transform(df_cleaned[numeric_columns])

# Verify results
print("Standardized Data:")
print(df_cleaned.head(20))


Standardized Data:
                                             car name          price  \
0                             Fiat 500e 2021 La Prima            NaN   
1                       Peugeot Traveller 2021 L3 VIP    SAR 140,575   
2                    Suzuki Jimny 2021 1.5L Automatic     SAR 98,785   
3                      Ford Bronco 2021 2.3T Big Bend    SAR 198,000   
4                       Honda HR-V 2021 1.8 i-VTEC LX            NaN   
5                       Honda HR-V 2021 1.8 i-VTEC EX     SAR 95,335   
6                      Peugeot Expert 2021 Van L3 A/T     SAR 82,845   
7                      Peugeot Expert 2021 Van L3 M/T     SAR 76,545   
8                   Renault Koleos 2021 2.5L LE (4WD)    SAR 116,900   
9                   Ford Bronco 2021 2.7T Outer Banks    SAR 238,000   
10                         Suzuki Jimny 2021 1.5L M/T     SAR 91,885   
11                      Honda HR-V 2021 1.8 i-VTEC DX     SAR 72,335   
12  Aston Martin DBS Superleggera Volante 202

In [ ]:
import numpy as np
import pandas as pd

# Assuming df_cleaned is already available (after preprocessing)
# Features and target selection
features = ['horse_power', 'engine_capacity', 'seats', 'top_speed']
target = 'price_in_usd'  # Updated to use the correct target column

# Prepare X (features) and y (target)
X = df_cleaned[features].values
y = df_cleaned[target].values

# Shuffle the data
np.random.seed(42)
indices = np.arange(len(X))
np.random.shuffle(indices)
X = X[indices]
y = y[indices]

# Split data manually (60% training, 20% validation, 20% testing)
n = len(X)
train_end = int(n * 0.6)
val_end = int(n * 0.8)

X_train, y_train = X[:train_end], y[:train_end]
X_val, y_val = X[train_end:val_end], y[train_end:val_end]
X_test, y_test = X[val_end:], y[val_end:]

# Print dataset sizes
print(f"Training set size: {len(X_train)}")
print(f"Validation set size: {len(X_val)}")
print(f"Test set size: {len(X_test)}")

# Standardize the features (manual scaling)
class StandardScaler:
    def fit(self, X):
        self.mean = np.mean(X, axis=0)
        self.std = np.std(X, axis=0)

    def transform(self, X):
        return (X - self.mean) / self.std

    def fit_transform(self, X):
        self.fit(X)
        return self.transform(X)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Ensure all data is in the correct format
X_train_scaled = np.array(X_train_scaled, dtype=float)
X_val_scaled = np.array(X_val_scaled, dtype=float)
X_test_scaled = np.array(X_test_scaled, dtype=float)

y_train = np.array(y_train, dtype=float).reshape(-1, 1)
y_val = np.array(y_val, dtype=float).reshape(-1, 1)
y_test = np.array(y_test, dtype=float).reshape(-1, 1)

Training set size: 3784
Validation set size: 1262
Test set size: 1262


In [ ]:
import numpy as np
import pandas as pd

# Assuming df_cleaned is already available (after preprocessing)
# Features and target selection
features = ['horse_power', 'engine_capacity', 'seats', 'top_speed']
target = 'price_in_usd'  # Updated to use the correct target column

# Prepare X (features) and y (target)
X = df_cleaned[features].values
y = df_cleaned[target].values

# Shuffle the data
np.random.seed(42)
indices = np.arange(len(X))
np.random.shuffle(indices)
X = X[indices]
y = y[indices]

# Split data manually (60% training, 20% validation, 20% testing)
n = len(X)
train_end = int(n * 0.6)
val_end = int(n * 0.8)

X_train, y_train = X[:train_end], y[:train_end]
X_val, y_val = X[train_end:val_end], y[train_end:val_end]
X_test, y_test = X[val_end:], y[val_end:]

# Print dataset sizes
print(f"Training set size: {len(X_train)}")
print(f"Validation set size: {len(X_val)}")
print(f"Test set size: {len(X_test)}")

# Standardize the features (manual scaling)
class StandardScaler:
    def fit(self, X):
        self.mean = np.mean(X, axis=0)
        self.std = np.std(X, axis=0)

    def transform(self, X):
        return (X - self.mean) / self.std

    def fit_transform(self, X):
        self.fit(X)
        return self.transform(X)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Ensure all data is in the correct format
X_train_scaled = np.array(X_train_scaled, dtype=float)
X_val_scaled = np.array(X_val_scaled, dtype=float)
X_test_scaled = np.array(X_test_scaled, dtype=float)

y_train = np.array(y_train, dtype=float).reshape(-1, 1)
y_val = np.array(y_val, dtype=float).reshape(-1, 1)
y_test = np.array(y_test, dtype=float).reshape(-1, 1)

# Linear Regression (Closed-form Solution)
def linear_regression_closed_form(X, y):
    X_b = np.c_[np.ones((X.shape[0], 1)), X]  # Add intercept
    theta = np.linalg.inv(X_b.T @ X_b) @ X_b.T @ y
    return theta

# Train the model on the training set (Linear Regression - Closed-form)
theta_closed_form = linear_regression_closed_form(X_train_scaled, y_train)
print(f"Linear Regression (Closed-form) coefficients: {theta_closed_form.ravel()}")

# Linear Regression (Gradient Descent)
def linear_regression_gradient_descent(X, y, learning_rate=0.01, n_iterations=1000):
    X_b = np.c_[np.ones((X.shape[0], 1)), X]  # Add intercept
    m = len(X_b)
    theta = np.zeros((X_b.shape[1], 1))

    for i in range(n_iterations):
        gradients = 2 / m * X_b.T @ (X_b @ theta - y)
        theta -= learning_rate * gradients

        if i % 100 == 0:
            cost = (1 / m) * np.sum((X_b @ theta - y) ** 2)
           # print(f"Iteration {i}: Cost = {cost}")
    return theta

# Train the model using gradient descent
theta_gd = linear_regression_gradient_descent(X_train_scaled, y_train, learning_rate=0.01, n_iterations=1000)
print(f"Linear Regression (Gradient Descent) coefficients: {theta_gd.ravel()}")

# Evaluate the models (Mean Squared Error)
def mean_squared_error_manual(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

# Evaluate closed-form model
X_b_train = np.c_[np.ones((X_train_scaled.shape[0], 1)), X_train_scaled]  # Add intercept term to training data
y_train_pred_closed_form = X_b_train @ theta_closed_form
mse_closed_form = mean_squared_error_manual(y_train, y_train_pred_closed_form)

# Evaluate gradient descent model
y_train_pred_gd = X_b_train @ theta_gd
mse_gradient_descent = mean_squared_error_manual(y_train, y_train_pred_gd)

# Display Mean Squared Errors
print(f"Training MSE (Closed-form): {mse_closed_form}")
print(f"Training MSE (Gradient Descent): {mse_gradient_descent}")

# Evaluate on validation data
X_b_val = np.c_[np.ones((X_val_scaled.shape[0], 1)), X_val_scaled]  # Add intercept term to validation data
y_val_pred_closed_form = X_b_val @ theta_closed_form
mse_val_closed_form = mean_squared_error_manual(y_val, y_val_pred_closed_form)

y_val_pred_gd = X_b_val @ theta_gd
mse_val_gradient_descent = mean_squared_error_manual(y_val, y_val_pred_gd)

print(f"Validation MSE (Closed-form): {mse_val_closed_form}")
print(f"Validation MSE (Gradient Descent): {mse_val_gradient_descent}")

# Ridge Regression (L2 Regularization)
def ridge_regression(X, y, lambda_):
    X_b = np.c_[np.ones((X.shape[0], 1)), X]  # Add intercept
    theta = np.linalg.inv(X_b.T @ X_b + lambda_ * np.eye(X_b.shape[1])) @ X_b.T @ y
    return theta

# Train Ridge Regression
lambda_ = 1.0  # Regularization strength
theta_ridge = ridge_regression(X_train_scaled, y_train, lambda_)
print(f"Ridge Regression coefficients: {theta_ridge.ravel()}")

# Evaluate Ridge Regression
y_train_pred_ridge = X_b_train @ theta_ridge
mse_ridge = mean_squared_error_manual(y_train, y_train_pred_ridge)
print(f"Training MSE (Ridge Regression): {mse_ridge}")

# LASSO Regression (L1 Regularization)
def lasso_regression(X, y, lambda_, n_iterations=1000, learning_rate=0.01):
    X_b = np.c_[np.ones((X.shape[0], 1)), X]  # Add intercept
    m = len(X_b)
    theta = np.zeros((X_b.shape[1], 1))

    for i in range(n_iterations):
        gradients = 2 / m * X_b.T @ (X_b @ theta - y)
        theta -= learning_rate * gradients
        # L1 regularization (shrinkage)
        theta[1:] = np.sign(theta[1:]) * np.maximum(0, np.abs(theta[1:]) - lambda_ * learning_rate)

    return theta

# Train LASSO Regression
lambda_ = 0.1  # Regularization strength
theta_lasso = lasso_regression(X_train_scaled, y_train, lambda_)
print(f"Lasso Regression coefficients: {theta_lasso.ravel()}")

# Evaluate LASSO Regression
y_train_pred_lasso = X_b_train @ theta_lasso
mse_lasso = mean_squared_error_manual(y_train, y_train_pred_lasso)
print(f"Training MSE (LASSO Regression): {mse_lasso}")

# Evaluate on validation data for Ridge and Lasso
X_b_val = np.c_[np.ones((X_val_scaled.shape[0], 1)), X_val_scaled]  # Add intercept term to validation data

# Ridge
y_val_pred_ridge = X_b_val @ theta_ridge
mse_val_ridge = mean_squared_error_manual(y_val, y_val_pred_ridge)
print(f"Validation MSE (Ridge Regression): {mse_val_ridge}")

# Lasso
y_val_pred_lasso = X_b_val @ theta_lasso
mse_val_lasso = mean_squared_error_manual(y_val, y_val_pred_lasso)
print(f"Validation MSE (Lasso Regression): {mse_val_lasso}")


Training set size: 3784
Validation set size: 1262
Test set size: 1262
Linear Regression (Closed-form) coefficients: [-0.02227897  0.31481752  0.06652657 -0.04874403  0.16935761]
Linear Regression (Gradient Descent) coefficients: [-0.02227897  0.3104884   0.0697642  -0.04890579  0.17109419]
Training MSE (Closed-form): 0.49012844522067084
Training MSE (Gradient Descent): 0.49013370462427847
Validation MSE (Closed-form): 0.3178616453937702
Validation MSE (Gradient Descent): 0.31799939566713964
Ridge Regression coefficients: [-0.02227308  0.31461186  0.06665721 -0.04875831  0.16938785]
Training MSE (Ridge Regression): 0.49012845851505366
Lasso Regression coefficients: [-0.02227897  0.31854172  0.01863558 -0.00300418  0.15395866]
Training MSE (LASSO Regression): 0.49535033684668794
Validation MSE (Ridge Regression): 0.3178742311605878
Validation MSE (Lasso Regression): 0.32858728644194996


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import numpy as np

# Initialize dictionaries to store results
results = {}

# Polynomial Regression: Evaluate for degrees from 2 to 10
for degree in range(2, 11):
    # Transform the features into polynomial features
    poly = PolynomialFeatures(degree=degree)
    X_train_poly = poly.fit_transform(X_train_scaled)
    X_val_poly = poly.transform(X_val_scaled)

    # Train the model
    model = LinearRegression()
    model.fit(X_train_poly, y_train)

    # Predict on validation set
    y_val_pred_poly = model.predict(X_val_poly)

    # Calculate MSE, MAE, and R² for Polynomial Regression
    mse_poly = mean_squared_error(y_val, y_val_pred_poly)
    mae_poly = mean_absolute_error(y_val, y_val_pred_poly)
    r2_poly = r2_score(y_val, y_val_pred_poly)

    # Store results in dictionary
    results[f"Polynomial Degree {degree}"] = {'MSE': mse_poly, 'MAE': mae_poly, 'R²': r2_poly}


# Linear Regression (no regularization)
linear_model = LinearRegression()
linear_model.fit(X_train_scaled, y_train)

# Predict on validation set
y_val_pred_lr = linear_model.predict(X_val_scaled)

# Calculate MSE, MAE, and R² for Linear Regression
mse_lr = mean_squared_error(y_val, y_val_pred_lr)
mae_lr = mean_absolute_error(y_val, y_val_pred_lr)
r2_lr = r2_score(y_val, y_val_pred_lr)

results["Linear Regression"] = {'MSE': mse_lr, 'MAE': mae_lr, 'R²': r2_lr}

print(f"Linear Regression - MSE: {mse_lr}, MAE: {mae_lr}, R²: {r2_lr}")

# Display the results in a better format
for model_name, metrics in results.items():
    print(f"\n{model_name}:")
    for metric, value in metrics.items():
        print(f"  {metric}: {value}")

# Optionally, you can plot the results for comparison
import matplotlib.pyplot as plt

# Extract MSE, MAE, R² for plotting
mse_values = [metrics['MSE'] for metrics in results.values()]
mae_values = [metrics['MAE'] for metrics in results.values()]
r2_values = [metrics['R²'] for metrics in results.values()]
model_names = list(results.keys())


# Compare models and select the best one based on MSE and R²
best_model = min(results, key=lambda model: results[model]['MSE'])
print(f"\nThe best model based on MSE is: {best_model}")

best_r2_model = max(results, key=lambda model: results[model]['R²'])
print(f"The best model based on R² is: {best_r2_model}")




Linear Regression - MSE: 0.3178616453937704, MAE: 0.28496842917623183, R²: 0.5169145346503181

Polynomial Degree 2:
  MSE: 0.25445871151858684
  MAE: 0.26665764286183924
  R²: 0.6132741812433651

Polynomial Degree 3:
  MSE: 0.41606437496977633
  MAE: 0.332136422192953
  R²: 0.36766623117204134

Polynomial Degree 4:
  MSE: 2.367518883336866
  MAE: 0.34667465460986385
  R²: -2.5981502583116125

Polynomial Degree 5:
  MSE: 48.40577318688776
  MAE: 0.4546183987843305
  R²: -72.56699307533731

Polynomial Degree 6:
  MSE: 475.3581722318252
  MAE: 0.9323791501161734
  R²: -721.4483581713886

Polynomial Degree 7:
  MSE: 146215.30221712778
  MAE: 12.690405639334985
  R²: -222216.71118469737

Polynomial Degree 8:
  MSE: 94230126158.08028
  MAE: 9900.185875896963
  R²: -143210749092.81815

Polynomial Degree 9:
  MSE: 1043326098017.8588
  MAE: 33439.07840150093
  R²: -1585644826533.6553

Polynomial Degree 10:
  MSE: 90610717707321.55
  MAE: 270779.64029769803
  R²: -137709979683404.9

Linear Regre

In [ ]:

import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso, Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR  # For RBF regression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Assuming df_cleaned is already available (after preprocessing)
# Features and target selection
features = ['horse_power', 'engine_capacity', 'seats', 'top_speed']
target = 'price_in_usd'

# Prepare X (features) and y (target)
X = df_cleaned[features].values
y = df_cleaned[target].values

# Shuffle the data
np.random.seed(42)
indices = np.arange(len(X))
np.random.shuffle(indices)
X = X[indices]
y = y[indices]

# Split data manually (60% training, 20% validation, 20% testing)
n = len(X)
train_end = int(n * 0.6)
val_end = int(n * 0.4)

X_train, y_train = X[:train_end], y[:train_end]
X_val, y_val = X[train_end:val_end], y[train_end:val_end]
X_test, y_test = X[val_end:], y[val_end:]

# Standardize the features (manual scaling)
class StandardScaler:
    def fit(self, X):
        self.mean = np.mean(X, axis=0)
        self.std = np.std(X, axis=0)

    def transform(self, X):
        return (X - self.mean) / self.std

    def fit_transform(self, X):
        self.fit(X)
        return self.transform(X)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

y_train = np.array(y_train, dtype=float).reshape(-1, 1)
y_val = np.array(y_val, dtype=float).reshape(-1, 1)
y_test = np.array(y_test, dtype=float).reshape(-1, 1)

# Linear Regression (Closed-form Solution)
def linear_regression_closed_form(X, y):
    X_b = np.c_[np.ones((X.shape[0], 1)), X]  # Add intercept
    theta = np.linalg.inv(X_b.T @ X_b) @ X_b.T @ y
    return theta

# Train the model on the training set
theta_closed_form = linear_regression_closed_form(X_train_scaled, y_train)
predictions_lr = np.c_[np.ones((X_val_scaled.shape[0], 1)), X_val_scaled] @ theta_closed_form

# Lasso Regression (L1 Regularization)
lasso = Lasso(alpha=0.1)
lasso.fit(X_train_scaled, y_train)
predictions_lasso = lasso.predict(X_val_scaled)

# Ridge Regression (L2 Regularization)
ridge = Ridge(alpha=0.1)
ridge.fit(X_train_scaled, y_train)
predictions_ridge = ridge.predict(X_val_scaled)

# Polynomial Regression (Degree 2 to 10)
poly_model = PolynomialFeatures(degree=3)
X_train_poly = poly_model.fit_transform(X_train_scaled)
X_val_poly = poly_model.transform(X_val_scaled)

poly_model = LinearRegression()
poly_model.fit(X_train_poly, y_train)
predictions_poly = poly_model.predict(X_val_poly)

# RBF Regression (using Support Vector Regression)
rbf_model = SVR(kernel='rbf')
rbf_model.fit(X_train_scaled, y_train.ravel())  # Flatten y_train
predictions_rbf = rbf_model.predict(X_val_scaled)

# Calculate MSE, MAE, and R² for each model
def evaluate_model(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return mse, mae, r2

# Evaluate all models
mse_lr, mae_lr, r2_lr = evaluate_model(y_val, predictions_lr)
mse_lasso, mae_lasso, r2_lasso = evaluate_model(y_val, predictions_lasso)
mse_ridge, mae_ridge, r2_ridge = evaluate_model(y_val, predictions_ridge)
mse_poly, mae_poly, r2_poly = evaluate_model(y_val, predictions_poly)
mse_rbf, mae_rbf, r2_rbf = evaluate_model(y_val, predictions_rbf)

# Print the results
print("Model Evaluation on Validation Set:")
print(f"Linear Regression (Closed-form): MSE={mse_lr}, MAE={mae_lr}, R²={r2_lr}")
print(f"LASSO Regression: MSE={mse_lasso}, MAE={mae_lasso}, R²={r2_lasso}")
print(f"Ridge Regression: MSE={mse_ridge}, MAE={mae_ridge}, R²={r2_ridge}")
print(f"Polynomial Regression (Degree 3): MSE={mse_poly}, MAE={mae_poly}, R²={r2_poly}")
print(f"RBF Regression: MSE={mse_rbf}, MAE={mae_rbf}, R²={r2_rbf}")

# Select the best model based on the lowest MSE and highest R²
models = {
    'Linear Regression': {'MSE': mse_lr, 'R²': r2_lr},
    'Lasso': {'MSE': mse_lasso, 'R²': r2_lasso},
    'Ridge': {'MSE': mse_ridge, 'R²': r2_ridge},
    'Polynomial Regression': {'MSE': mse_poly, 'R²': r2_poly},
    'RBF Regression': {'MSE': mse_rbf, 'R²': r2_rbf}
}

# Find the model with the lowest MSE
best_model_mse = min(models, key=lambda x: models[x]['MSE'])
best_model_r2 = max(models, key=lambda x: models[x]['R²'])

print(f"Best Model based on MSE: {best_model_mse}")
print(f"Best Model based on R²: {best_model_r2}")

# Test the best model on the test set (optional)
# You can use the best model (either MSE or R²) and evaluate it on the test set

ValueError: Found array with 0 sample(s) (shape=(0, 4)) while a minimum of 1 is required by Lasso.

In [21]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso, Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR  # For RBF regression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Assuming df_cleaned is already available (after preprocessing)
# Features and target selection
features = ['horse_power', 'engine_capacity', 'seats', 'top_speed']
target = 'price_in_usd'

# Prepare X (features) and y (target)
X = df_cleaned[features].values
y = df_cleaned[target].values

# Shuffle the data
np.random.seed(42)
indices = np.arange(len(X))
np.random.shuffle(indices)
X = X[indices]
y = y[indices]

# Split data manually (60% training, 20% validation, 20% testing)
n = len(X)
train_end = int(n * 0.6)
val_end = int(n * 0.8)

X_train, y_train = X[:train_end], y[:train_end]
X_val, y_val = X[train_end:val_end], y[train_end:val_end]
X_test, y_test = X[val_end:], y[val_end:]

# Standardize the features (manual scaling)
class StandardScaler:
    def fit(self, X):
        self.mean = np.mean(X, axis=0)
        self.std = np.std(X, axis=0)

    def transform(self, X):
        return (X - self.mean) / self.std

    def fit_transform(self, X):
        self.fit(X)
        return self.transform(X)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

y_train = np.array(y_train, dtype=float).reshape(-1, 1)
y_val = np.array(y_val, dtype=float).reshape(-1, 1)
y_test = np.array(y_test, dtype=float).reshape(-1, 1)

# Linear Regression (Closed-form Solution)
def linear_regression_closed_form(X, y):
    X_b = np.c_[np.ones((X.shape[0], 1)), X]  # Add intercept
    theta = np.linalg.inv(X_b.T @ X_b) @ X_b.T @ y
    return theta

# Train the model on the training set
theta_closed_form = linear_regression_closed_form(X_train_scaled, y_train)
predictions_lr = np.c_[np.ones((X_val_scaled.shape[0], 1)), X_val_scaled] @ theta_closed_form

# Lasso Regression (L1 Regularization)
lasso = Lasso(alpha=0.1)
lasso.fit(X_train_scaled, y_train)
predictions_lasso = lasso.predict(X_val_scaled)

# Ridge Regression (L2 Regularization)
ridge = Ridge(alpha=0.1)
ridge.fit(X_train_scaled, y_train)
predictions_ridge = ridge.predict(X_val_scaled)

# Polynomial Regression (Degree 2 to 10)
poly_model = PolynomialFeatures(degree=3)
X_train_poly = poly_model.fit_transform(X_train_scaled)
X_val_poly = poly_model.transform(X_val_scaled)

poly_model = LinearRegression()
poly_model.fit(X_train_poly, y_train)
predictions_poly = poly_model.predict(X_val_poly)

# RBF Regression (using Support Vector Regression)
rbf_model = SVR(kernel='rbf')
rbf_model.fit(X_train_scaled, y_train.ravel())  # Flatten y_train
predictions_rbf = rbf_model.predict(X_val_scaled)

# Calculate MSE, MAE, and R² for each model
def evaluate_model(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return mse, mae, r2

# Evaluate all models
mse_lr, mae_lr, r2_lr = evaluate_model(y_val, predictions_lr)
mse_lasso, mae_lasso, r2_lasso = evaluate_model(y_val, predictions_lasso)
mse_ridge, mae_ridge, r2_ridge = evaluate_model(y_val, predictions_ridge)
mse_poly, mae_poly, r2_poly = evaluate_model(y_val, predictions_poly)
mse_rbf, mae_rbf, r2_rbf = evaluate_model(y_val, predictions_rbf)

# Print the results
print("Model Evaluation on Validation Set:")
print(f"Linear Regression (Closed-form): MSE={mse_lr}, MAE={mae_lr}, R²={r2_lr}")
print(f"LASSO Regression: MSE={mse_lasso}, MAE={mae_lasso}, R²={r2_lasso}")
print(f"Ridge Regression: MSE={mse_ridge}, MAE={mae_ridge}, R²={r2_ridge}")
print(f"Polynomial Regression (Degree 3): MSE={mse_poly}, MAE={mae_poly}, R²={r2_poly}")
print(f"RBF Regression: MSE={mse_rbf}, MAE={mae_rbf}, R²={r2_rbf}")

# Select the best model based on the lowest MSE and highest R²
models = {
    'Linear Regression': {'MSE': mse_lr, 'R²': r2_lr},
    'Lasso': {'MSE': mse_lasso, 'R²': r2_lasso},
    'Ridge': {'MSE': mse_ridge, 'R²': r2_ridge},
    'Polynomial Regression': {'MSE': mse_poly, 'R²': r2_poly},
    'RBF Regression': {'MSE': mse_rbf, 'R²': r2_rbf}
}

# Find the model with the lowest MSE
best_model_mse = min(models, key=lambda x: models[x]['MSE'])
best_model_r2 = max(models, key=lambda x: models[x]['R²'])

print(f"Best Model based on MSE: {best_model_mse}")
print(f"Best Model based on R²: {best_model_r2}")

# Test the best model on the test set (optional)
# You can use the best model (either MSE or R²) and evaluate it on the test set


Model Evaluation on Validation Set:
Linear Regression (Closed-form): MSE=0.3178616453937702, MAE=0.28496842917623194, R²=0.5169145346503183
LASSO Regression: MSE=0.34577299683232354, MAE=0.2883100367399289, R²=0.47449492097994783
Ridge Regression: MSE=0.3178628960810628, MAE=0.28496852953088825, R²=0.5169126338583812
Polynomial Regression (Degree 3): MSE=0.41606437496977633, MAE=0.332136422192953, R²=0.36766623117204134
RBF Regression: MSE=0.2159463066710089, MAE=0.22073166149325918, R²=0.67180525376229
Best Model based on MSE: RBF Regression
Best Model based on R²: RBF Regression


In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Assuming X_train_scaled, y_train, X_val_scaled, y_val are already available
# Feature names (corresponding to columns of X_train_scaled)
feature_names = ['horse_power', 'engine_capacity', 'seats', 'top_speed']

# Forward Selection Implementation
def forward_selection(X_train, y_train, X_val, y_val, feature_names, max_features=None):
    selected_features = []
    remaining_features = list(range(X_train.shape[1]))
    model_performance = []

    while remaining_features and (max_features is None or len(selected_features) < max_features):
        best_feature = None
        best_mse = float('inf')

        # Test adding each remaining feature
        for feature in remaining_features:
            trial_features = selected_features + [feature]
            X_train_subset = X_train[:, trial_features]
            X_val_subset = X_val[:, trial_features]

            # Train the model on the subset of features
            model = LinearRegression()
            model.fit(X_train_subset, y_train)
            predictions = model.predict(X_val_subset)

            # Calculate MSE on validation set
            mse = mean_squared_error(y_val, predictions)

            # Check if this feature improves the performance
            if mse < best_mse:
                best_mse = mse
                best_feature = feature

        if best_feature is not None:
            # Add the best feature to the selected features
            selected_features.append(best_feature)
            remaining_features.remove(best_feature)
            model_performance.append(best_mse)
        else:
            break  # Stop if no feature improves the performance

    # Map selected feature indices back to feature names
    selected_feature_names = [feature_names[i] for i in selected_features]

    return selected_feature_names, model_performance


# Perform forward selection
selected_features, performance = forward_selection(
    X_train=X_train_scaled,
    y_train=y_train,
    X_val=X_val_scaled,
    y_val=y_val,
    feature_names=feature_names,
    max_features=3  # Optional: Limit to 3 features
)

# Display the results
print("Selected Features:", selected_features)
print("Performance (MSE at each step):", performance)


Selected Features: ['horse_power', 'top_speed', 'seats']
Performance (MSE at each step): [0.34679840070516454, 0.318982533645658, 0.3181483623101357]


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Lasso, Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# Assuming df_cleaned is already available (after preprocessing)
# Features and target selection
features = ['horse_power', 'engine_capacity', 'seats', 'top_speed']
target = 'price_in_usd'

# Prepare X (features) and y (target)
X = df_cleaned[features].values
y = df_cleaned[target].values

# Ensure the number of samples in X and y are consistent
if X.shape[0] != y.shape[0]:
    raise ValueError(f"Number of samples in X ({X.shape[0]}) does not match number of samples in y ({y.shape[0]})")

# Split the data (80% train, 20% test)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features (scaling)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# ----- LASSO REGRESSION -----
lasso = Lasso()

# Set up a range of lambda values (alpha in sklearn is the regularization strength)
lasso_params = {'alpha': np.logspace(-4, 1, 50)}  # Grid search over different alpha values

# Perform GridSearchCV for LASSO
lasso_grid = GridSearchCV(lasso, lasso_params, cv=5, scoring='neg_mean_squared_error')
lasso_grid.fit(X_train_scaled, y_train)

# Get the best alpha (regularization strength) for LASSO
best_lasso_alpha = lasso_grid.best_params_['alpha']
print(f"Best LASSO alpha: {best_lasso_alpha}")

# Evaluate the model on the validation set
lasso_best = lasso_grid.best_estimator_
y_pred_lasso = lasso_best.predict(X_val_scaled)
lasso_mse = mean_squared_error(y_val, y_pred_lasso)
print(f"LASSO Mean Squared Error: {lasso_mse}")

# ----- RIDGE REGRESSION -----
ridge = Ridge()

# Set up a range of lambda values (alpha in sklearn is the regularization strength)
ridge_params = {'alpha': np.logspace(-4, 1, 50)}  # Grid search over different alpha values

# Perform GridSearchCV for Ridge
ridge_grid = GridSearchCV(ridge, ridge_params, cv=5, scoring='neg_mean_squared_error')
ridge_grid.fit(X_train_scaled, y_train)

# Get the best alpha (regularization strength) for Ridge
best_ridge_alpha = ridge_grid.best_params_['alpha']
print(f"Best Ridge alpha: {best_ridge_alpha}")

# Evaluate the model on the validation set
ridge_best = ridge_grid.best_estimator_
y_pred_ridge = ridge_best.predict(X_val_scaled)
ridge_mse = mean_squared_error(y_val, y_pred_ridge)
print(f"Ridge Mean Squared Error: {ridge_mse}")

Best LASSO alpha: 0.0001
LASSO Mean Squared Error: 0.6642932989279161
Best Ridge alpha: 10.0
Ridge Mean Squared Error: 0.6642947761911823


In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

# Assuming the best model is Lasso (for example)
# But you can similarly do it for Ridge or any other model based on validation performance.

# Train the Lasso model on the entire training set using the best alpha
lasso_best = lasso_grid.best_estimator_

# Now, evaluate it on the test set (X_test_scaled and y_test are assumed to be available)
y_pred_test_lasso = lasso_best.predict(X_test_scaled)

# Calculate MSE and R² for the test set
lasso_test_mse = mean_squared_error(y_test, y_pred_test_lasso)
lasso_test_r2 = r2_score(y_test, y_pred_test_lasso)

# Display the performance on the test set
print(f"Lasso Test Set Performance:")
print(f"Test MSE: {lasso_test_mse}")
print(f"Test R²: {lasso_test_r2}")

# Similarly, you can evaluate the Ridge model if it performed better
ridge_best = ridge_grid.best_estimator_
y_pred_test_ridge = ridge_best.predict(X_test_scaled)

# Calculate MSE and R² for the test set
ridge_test_mse = mean_squared_error(y_test, y_pred_test_ridge)
ridge_test_r2 = r2_score(y_test, y_pred_test_ridge)

# Display the performance on the test set for Ridge
print(f"Ridge Test Set Performance:")
print(f"Test MSE: {ridge_test_mse}")
print(f"Test R²: {ridge_test_r2}")


Lasso Test Set Performance:
Test MSE: 1.6918179804712496
Test R²: 0.19573985639353386
Ridge Test Set Performance:
Test MSE: 1.6918633212101688
Test R²: 0.19571830221358333
